In [1]:
# import save_load as sl
# import preprocess_data as ppd
import torch
from torch import nn
from torch.utils.data import DataLoader
# from model import DnCNN
# from Dataset import Img_Dataset
import numpy as np 
import pathlib

# Importing utitility functions for training
from PT_files.model import DnCNN_2k, DnCNN_B, DnCNN
from PT_files.Dataset import Img_Dataset, Large_Img_Dataset
import PT_files.preprocess_data as ppd
import PT_files.save_load as sl
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

Using cuda device


In [2]:
# data_6k = sl.NERSC_load('test_data70-6000.npy')
data_6k = sl.NERSC_load('test_data_40%_6000.npy')

Quick test to see if running inference on the CPU allows for inference on the entire 6k by 6k image to be performed

In [3]:
def full_grid_pass_window(dataset, model, model_params, samp_idx):
    
    full = np.empty((1, 1, 6000,6000))
    count = np.empty((1, 1, 6000,6000))
    
    noise_data = dataset[0]
    param_name = model_params
    print('pass 1')
    
    current_dir = pathlib.Path().resolve()
    model_params_path = current_dir / 'Model_params'
    assert model_params_path.exists()
    model_path = model_params_path / param_name
    print('pass 2')
    
    model = model()
    model.to(device)
    model.load_state_dict(torch.load(str(model_path)))
    model.eval();
    print('pass 3')
    # telling pytorch this is for inference and not learning, so keeps the weights unchanged
    with torch.no_grad():
        
        print('pass 4')
        torch.cuda.empty_cache()
        test_noise = torch.as_tensor(noise_data[samp_idx:samp_idx+1,:,:, :])
        test_noise = test_noise.to(device)

        print('pass 5')
        output = model(test_noise)
        print('pass 6')
        resid_img = output.to('cpu').detach().numpy()
        print('pass 7')

        full[:, :, :, :] += resid_img

    return full

Test of model trained on batch size of 16, patch size of 150x150, and 200 epochs

In [4]:
# full = full_grid_pass_window(dataset=data_6k,
#                              model=DnCNN_B,
#                              model_params="6k_model_wb_e800.pth",
#                              samp_idx=0)

Time for cpu based inference for base DnCNN w adam optimizer 

In [5]:
full = full_grid_pass_window(dataset=data_6k, model=DnCNN_2k, model_params="6k_model_wb_e800_lys10_32feat.pth", samp_idx=0)

pass 1
pass 2
pass 3
pass 4
pass 5
pass 6
pass 7


In [6]:
# fig, ax = plt.subplots(1, 1, figsize=(24,20))
# vmin, vmax = np.percentile(data_6k[0][0][0], (1,99))

# plt.imshow(full[0][0][2800:3000, 2800:3000], vmin=vmin, vmax=vmax)

In [7]:
sl.NERSC_save('6k_model_wb_e800_lys10_32feat.npy', full)